In [221]:
#ICT305 Assignment 1

In [222]:
#Imports and Setup

In [223]:
#Imports
import os
import pandas as pd
import numpy as np
import datetime
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
import import_ipynb

from bokeh.core.enums import Align
from bokeh.core.properties import Enum
from bokeh.io import curdoc, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, ColumnDataSource, Div, Slider, Range1d, PrintfTickFormatter, InlineStyleSheet, Button, CustomJS, SetValue, TabPanel, Tabs, Dropdown
from bokeh.palettes import Spectral6
from bokeh.layouts import column, row

import_ipynb.NotebookLoader("./AssigOne_Temps.ipynb")

from src.other_graphs.evap_graph import *
from src.other_graphs.rain_graph import *
from src.other_graphs.sun_graph import *
from src.Utils import *

# Load title HTML file
with open('./python/src/titleDiv.html', 'r', encoding='utf-8') as f:
    titleDivContents = f.read()

# Define the folder path
folder_path = "./python/ICT305_States"

# Create DFs
adelaideDF = pd.read_csv(os.path.join(folder_path, "Adelaide.csv"), encoding='latin1')
brisbaneDF = pd.read_csv(os.path.join(folder_path, "Brisbane.csv"), encoding='latin1')
darwinDF = pd.read_csv(os.path.join(folder_path, "Darwin.csv"), encoding='latin1')
hobartDF = pd.read_csv(os.path.join(folder_path, "Hobart.csv"), encoding='latin1')
perthDF = pd.read_csv(os.path.join(folder_path, "Perth.csv"), encoding='latin1')
melbourneDF = pd.read_csv(os.path.join(folder_path, "Melbourne.csv"), encoding='latin1')
sydneyDF = pd.read_csv(os.path.join(folder_path, "Sydney.csv"), encoding='latin1')

# Rename columns because they have garbage ascii characters in their names
new_columns = {
    'ï»¿Date': 'Date',
    'Minimum temperature (ÃÂ°C)': 'Min Temperature (°C)',
    'Maximum temperature (ÃÂ°C)': 'Max Temperature (°C)',
    'Rainfall (mm)': 'Rainfall (mm)',
    'Evaporation (mm)': 'Evaporation (mm)',
    'Sunshine (hours)': 'Sunshine (hours)',
    'Direction of maximum wind gust ': 'Max Wind Gust Direction',
    'Speed of maximum wind gust (km/h)': 'Max Wind Gust Speed (km/h)',
    'Time of maximum wind gust': 'Time of Max Wind Gust',
    '9am Temperature (ÃÂ°C)': '9am Temperature (°C)',
    '9am relative humidity (%)': '9am Relative Humidity (%)',
    '9am cloud amount (oktas)': '9am Cloud Amount (oktas)',
    '9am wind direction': '9am Wind Direction',
    '9am wind speed (km/h)': '9am Wind Speed (km/h)',
    '9am MSL pressure (hPa)': '9am MSL Pressure (hPa)',
    '3pm Temperature (ÃÂ°C)': '3pm Temperature (°C)',
    '3pm relative humidity (%)': '3pm Relative Humidity (%)',
    '3pm cloud amount (oktas)': '3pm Cloud Amount (oktas)',
    '3pm wind direction': '3pm Wind Direction',
    '3pm wind speed (km/h)': '3pm Wind Speed (km/h)',
    '3pm MSL pressure (hPa)': '3pm MSL Pressure (hPa)'
}

# Rename columns in each DataFrame
adelaideDF = adelaideDF.rename(columns=new_columns)
brisbaneDF = brisbaneDF.rename(columns=new_columns)
darwinDF = darwinDF.rename(columns=new_columns)
hobartDF = hobartDF.rename(columns=new_columns)
perthDF = perthDF.rename(columns=new_columns)
melbourneDF = melbourneDF.rename(columns=new_columns)
sydneyDF = sydneyDF.rename(columns=new_columns)
# Adding the City to each frame
adelaideDF.insert(1, "city", "adelaide")
brisbaneDF.insert(1, "city", "brisbane")
darwinDF.insert(1, "city", "darwin")
hobartDF.insert(1, "city", "hobart")
perthDF.insert(1, "city", "perth")
melbourneDF.insert(1, "city", "melbourne")
sydneyDF.insert(1, "city", "sydney")

# Grouping the data
dataframes = [adelaideDF, brisbaneDF, darwinDF, hobartDF, melbourneDF, perthDF, sydneyDF]
all_cities_df = pd.concat(dataframes)

#Pre-process and update the dates of the dataframe to be the correct format
all_cities_df["Date"] = pd.to_datetime(all_cities_df["Date"], format='%d/%m/%Y')


In [224]:
#Pre-processing Data

In [231]:
# Add day, month and year columns
all_cities_df['day'] = all_cities_df['Date'].dt.day
all_cities_df['month'] = all_cities_df['Date'].dt.month
all_cities_df['year'] = all_cities_df['Date'].dt.year

#Trim Data
df_trim = all_cities_df.loc[:,('Date', 'city', 'day', 'month', 'year', 'Evaporation (mm)', 'Rainfall (mm)', 'Sunshine (hours)')]
df_trim.columns = ['date', 'city', 'day', 'month', 'year', 'evap', 'rain', 'sun']
df_trim.fillna(value=0.0, inplace=True)
df_trim.to_csv(path_or_buf="all_cities.csv")

# Default values
default_capital = "perth"
default_year = min(df_trim.year[df_trim.city == default_capital])
default_month = min(df_trim.month[df_trim.year == default_year])

# Default Data trimmed to default values
data = df_trim[(df_trim['year'] == default_year) & (df_trim['month'] == default_month) & (df_trim['city'] == default_capital)]


['adelaide' 'brisbane' 'darwin' 'hobart' 'melbourne' 'perth' 'sydney']


In [226]:
#Plot Setup

In [227]:
#Setup plot source data and build plots
trimmedSourceData = ColumnDataSource(data={
    'x' : data.day,
    'evap' : data.evap,
    'rain' : data.rain,
    'sun' : data.sun,
    'city' : data.city
})

evapPlot = CreateLinePlot("Perth Evaporation in {month} {year}", df_trim, trimmedSourceData, default_month, default_year, "x", "evap", "Day of the Month", "Evaporation")
rainPlot = CreateLinePlot("Perth Rainfall in {month} {year}", df_trim, trimmedSourceData, default_month, default_year,"x", "rain", "Day of the Month", "Rainfall")
sunPlot = CreateLinePlot("Perth Sunshine in {month} {year}", df_trim, trimmedSourceData, default_month, default_year, "x", "sun", "Day of the Month", "Sunshine")


In [228]:
#Plot Layouts
year_slider = Slider(start=min(df_trim.year), end=max(df_trim.year), step=1, value=default_year, title='Year')
month_slider = Slider(start=min(df_trim.month[(df_trim.year == year_slider.value) & (df_trim.city == default_capital)]), end=max(df_trim.month[(df_trim.year == year_slider.value) & (df_trim.city == default_capital)]), step=1, value=min(df_trim.month[(df_trim.year == year_slider.value) & (df_trim.city == default_capital)]), title='Month')

dropdownMenu = [("Perth", 'perth'), ("Adelaide", "adelaide"), ("Brisbane", "brisbane"), ("Darwin", "darwin"), ("Hobart", "hobart"), ("Melbourne", "melbourne"), ("Sydney", "sydney"), ("All", "all")]
capitalDropdown = Dropdown(label="Capital City", button_type="success", menu=dropdownMenu)

currentlySelectedCapital = default_capital

# Creating the hover tools for the plot
evapPlot.add_tools(CreateLinePlotHoverTool(('Evaporation', '@evap{%smm}'),[('Rainfall', '@rain{%smm}'), ('Sunshine', '@sun{%s hours}'), ('City', '@city')]))
rainPlot.add_tools(CreateLinePlotHoverTool(('Rainfall', '@rain{%smm}'),[('Evaporation', '@evap{%smm}'), ('Sunshine', '@sun{%s hours}'), ('City', '@city')]))
sunPlot.add_tools(CreateLinePlotHoverTool(('Sunshine', '@sun{%s hours}'),[('Rainfall', '@rain{%smm}'), ('Evaporation', '@evap{%smm}'), ('City', '@city')]))

def update_plots(yearValue, monthValue, capital):
    global trimmedSourceData, evapPlot, rainPlot, sunPlot
    #Check if all cities are being plotted
    if(capital == 'all'):
        d = df_trim[ (df_trim['year'] == yearValue) & (df_trim['month'] == monthValue)].reset_index(drop=True)
    else:
        d = df_trim[ (df_trim['year'] == yearValue) & (df_trim['month'] == monthValue) & (df_trim['city'] == capital)].reset_index(drop=True)
    
    #Only update the data of the plot if there's data to use
    if not d.empty:
        newData = {
            'x' : d.day,
            'evap' : d.evap,
            'rain' : d.rain,
            'sun' : d.sun,
            'city' : d.city
        }
        
        trimmedSourceData.data = dict(newData)
    
        yEvapmin, yEvapmax = GetMinMax(d['evap'])
        yRainmin, yRainmax = GetMinMax(d['rain'])
        ySunmin, ySunmax = GetMinMax(d['sun'])
        
        if not capital == 'all':
            evapPlot.y_range.start = yEvapmin - 1
            evapPlot.y_range.end = yEvapmax + 5
            rainPlot.y_range.start = yRainmin - 1
            rainPlot.y_range.end = yRainmax + 5
            sunPlot.y_range.start = ySunmin - 1
            sunPlot.y_range.end= ySunmax + 5
        else:
            del(evapPlot)
            del(rainPlot)
            del(sunPlot)
            
            evapPlot = CreateMultiLinePlot("All Capitals Evaporation in {month} {year}", df_trim, trimmedSourceData, monthValue, yearValue, "x", "evap", "Day of the Month", "Evaporation")
            rainPlot = CreateMultiLinePlot("All Capitals Rainfall in {month} {year}", df_trim, trimmedSourceData, monthValue, yearValue, "x", "rain", "Day of the Month", "Rainfall")
            sunPlot = CreateMultiLinePlot("All Capitals Sunshine in {month} {year}", df_trim, trimmedSourceData, monthValue, yearValue, "x", "sun", "Day of the Month", "Sunshine")
            
            # Creating the hover tools for the plot
            evapPlot.add_tools(CreateLinePlotHoverTool(('Evaporation', '@evap{%smm}'),[('Rainfall', '@rain{%smm}'), ('Sunshine', '@sun{%s hours}'), ('City', '@city')]))
            rainPlot.add_tools(CreateLinePlotHoverTool(('Rainfall', '@rain{%smm}'),[('Evaporation', '@evap{%smm}'), ('Sunshine', '@sun{%s hours}'), ('City', '@city')]))
            sunPlot.add_tools(CreateLinePlotHoverTool(('Sunshine', '@sun{%s hours}'),[('Rainfall', '@rain{%smm}'), ('Evaporation', '@evap{%smm}'), ('City', '@city')]))
            
    else:
        newData = {
            'x' : [0],
            'evap' : [0],
            'rain' : [0],
            'sun' : [0],
            'city' : currentlySelectedCapital
        }
        
        trimmedSourceData.data = dict(newData)
    
    
    if(capital == "all"):
        evapPlot.title.text = "All Capitals Evaporation in {month} {year}".format(month = datetime.datetime.strptime(str(monthValue), '%m').strftime('%B'), year = yearValue)
        rainPlot.title.text = "All Capitals Rainfall in {month} {year}".format(month = datetime.datetime.strptime(str(monthValue), '%m').strftime('%B'), year = yearValue)
        sunPlot.title.text = "All Capitals Sunshine in {month} {year}".format(month = datetime.datetime.strptime(str(monthValue), '%m').strftime('%B'), year = yearValue)
    else:
        evapPlot.title.text = currentlySelectedCapital.capitalize() + " Evaporation in {month} {year}".format(month = datetime.datetime.strptime(str(monthValue), '%m').strftime('%B'), year = yearValue)
        rainPlot.title.text = currentlySelectedCapital.capitalize() + " Rainfall in {month} {year}".format(month = datetime.datetime.strptime(str(monthValue), '%m').strftime('%B'), year = yearValue)
        sunPlot.title.text = currentlySelectedCapital.capitalize() + " Sunshine in {month} {year}".format(month = datetime.datetime.strptime(str(monthValue), '%m').strftime('%B'), year = yearValue)
    
    #month_slider.end = max(df_trim.month[df_trim.year == year_slider.value])
    #if month_slider.end < month_slider.value:
    #    month_slider.value = month_slider.end

def slidersOnChange(attr, old, new):
    global year_slider, month_slider, currentlySelectedCapital
    clampSliders()
    update_plots(year_slider.value, month_slider.value, currentlySelectedCapital)

def capitalDropdownOnClick(event):
    global year_slider, month_slider, currentlySelectedCapital
    currentlySelectedCapital = event.item
    clampSliders()

    update_plots(year_slider.value, month_slider.value, currentlySelectedCapital)

def clampSliders():
    global year_slider, month_slider, currentlySelectedCapital, df_trim
    # Get the new max year for the currently selected city
    if(currentlySelectedCapital == 'all'):
        yMin, yMax = GetMinMax(df_trim.year)
    else:
        yMin, yMax = GetMinMax(df_trim.year[df_trim.city == currentlySelectedCapital])
    # Clamp the year value incase the selected value is outside the range
    year_slider.value = clamp(year_slider.value, yMin, yMax)
    # Set new min and max year values
    year_slider.start = yMin
    year_slider.end = yMax
    # Get the new max month for the currently selected city
    if(currentlySelectedCapital == 'all'):
        mMin, mMax = GetMinMax(df_trim.month[(df_trim.year == year_slider.value)])
    else:
        mMin, mMax = GetMinMax(df_trim.month[(df_trim.city == currentlySelectedCapital) & (df_trim.year == year_slider.value)])
    # Set new min and max month values
    month_slider.start = mMin
    month_slider.end = mMax
    #Clamp the month value incase the selected value is outside the range
    month_slider.value = clamp(month_slider.value, mMin, mMax)
   
year_slider.on_change('value', slidersOnChange)
month_slider.on_change('value', slidersOnChange)
capitalDropdown.on_click(capitalDropdownOnClick)

titleDiv = Div(text=titleDivContents)

In [229]:
# Temperature Graphs

In [230]:
#TODO - Make temperatures have sub-tabs
janTempsTab = TabPanel(child=AssigOne_Temps.janTemps, title='January 2021')
monthlyPlotTempsTab = TabPanel(child=AssigOne_Temps.monthly_Plot_layout, title='Monthly')
monthlyAvgPlotTempsTab = TabPanel(child=AssigOne_Temps.monthly_avg_temp_layout, title='Monthly Avg Temperatures')
temp_YearOverYear_plotTempsTab = TabPanel(child=AssigOne_Temps.temp_YearOverYear_plot_layout, title='Year vs Year')
monthly_MinMax_plot_layoutTempsTab = TabPanel(child=AssigOne_Temps.monthly_MinMax_plot_layout, title='Monthly Min/Max')
yearly_MinMax_plot_layoutTempsTab = TabPanel(child=AssigOne_Temps.yearly_MinMax_plot_layout, title='Yearly Min/Max')

tempTabs = TabPanel(child=Tabs(tabs=[janTempsTab,monthlyPlotTempsTab,monthlyAvgPlotTempsTab,temp_YearOverYear_plotTempsTab,monthly_MinMax_plot_layoutTempsTab,yearly_MinMax_plot_layoutTempsTab], margin=(10,0,0,0)), title='Temperature')

evapTab = CreateGraphTabPanel(capitalDropdown, year_slider, month_slider, evapPlot, "Evaporation")
rainTab = CreateGraphTabPanel(capitalDropdown, year_slider, month_slider, rainPlot, "Rainfall")
sunTab = CreateGraphTabPanel(capitalDropdown, year_slider, month_slider, sunPlot, "Sunshine")

layout = CreatePageLayout(titleItem=titleDiv, pageTabItems=[tempTabs, evapTab, rainTab, sunTab], align="center", margin=(50, 0, 50, 50))

curdoc().add_root(layout)
curdoc().theme = 'dark_minimal'
curdoc().title = "ICT305 Assignment 2 - Pink Fluffy Unicorns"
